In [1]:
!pip install lightgbm --quiet
!pip install category_encoders --quiet
!pip install pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.5 MB/s eta 0:00:00


In [2]:
# 1) Librerías
import pandas as pd
import numpy as np
from datetime import timedelta
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import files

In [3]:
# 2) Cargar archivos subidos
df1 = pd.read_csv("2023.csv")
df2 = pd.read_csv("2024.csv")

In [4]:
# 3) Unir
df = pd.concat([df1, df2], ignore_index=True).copy()

In [6]:
df.head()

,Date,Source,Site ID,POC,Daily Max 8-hour CO Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,...,AQS Parameter Description,Method Code,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude
0,01/01/2023,AQS,360050133,1,0.4,ppm,5,PFIZER LAB SITE,19,79.0,...,Carbon monoxide,593,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.8679,-73.87809
1,01/02/2023,AQS,360050133,1,0.4,ppm,5,PFIZER LAB SITE,24,100.0,...,Carbon monoxide,593,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.8679,-73.87809
2,01/03/2023,AQS,360050133,1,0.8,ppm,9,PFIZER LAB SITE,18,75.0,...,Carbon monoxide,593,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.8679,-73.87809
3,01/04/2023,AQS,360050133,1,0.8,ppm,9,PFIZER LAB SITE,24,100.0,...,Carbon monoxide,593,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.8679,-73.87809
4,01/05/2023,AQS,360050133,1,0.3,ppm,3,PFIZER LAB SITE,24,100.0,...,Carbon monoxide,593,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.8679,-73.87809


In [7]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6939 entries, 0 to 6938
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Date                               6939 non-null   object 
 1   Source                             6939 non-null   object 
 2   Site ID                            6939 non-null   int64  
 3   POC                                6939 non-null   int64  
 4   Daily Max 8-hour CO Concentration  6939 non-null   float64
 5   Units                              6939 non-null   object 
 6   Daily AQI Value                    6939 non-null   int64  
 7   Local Site Name                    6939 non-null   object 
 8   Daily Obs Count                    6939 non-null   int64  
 9   Percent Complete                   6939 non-null   float64
 10  AQS Parameter Code                 6939 non-null   int64  
 11  AQS Parameter Description          6939 non-null   objec

In [5]:
# 4) Normalizar nombres (por si vienen con espacios extra)
df.columns = [c.strip() for c in df.columns]

In [8]:
# 5) Asegurar formato de fecha
df['Date'] = pd.to_datetime(df['Date'], dayfirst=False, errors='coerce')

In [9]:
# 6) Renombrar columna objetivo a 'CO' si viene con nombre largo
# Busca columna que contenga 'Daily Max' y 'CO' (según lo compartido)
target_cols = [c for c in df.columns if 'Daily Max' in c and 'CO' in c]
if len(target_cols)==0:
    raise ValueError("No encontré la columna objetivo. Revisa los nombres de columnas.")
target_col = target_cols[0]
df = df.rename(columns={target_col: 'CO'})

In [10]:
# 7) Ordenar y reset index
df = df.sort_values('Date').reset_index(drop=True)

In [11]:
# 8) Imputar/limpiar CO y columnas numéricas básicas
df['CO'] = pd.to_numeric(df['CO'], errors='coerce')

In [12]:
# Opcional: eliminar filas sin fecha
df = df[~df['Date'].isna()].copy()

In [13]:
# 9) Asegurar que para cada site/date haya solo una fila.
# Si tienes múltiples registros por estación/día, podrías agrupar por mean:
group_cols = ['Date','Site ID']
if 'Site ID' not in df.columns:
    # si no existe site id, creamos una sola 'site'
    df['Site ID'] = 'ALL'
else:
    # puede tener espacios/luego normalizamos:
    df['Site ID'] = df['Site ID'].astype(str).str.strip()

df = df.groupby(group_cols, as_index=False).agg({
    'CO':'mean',
    # mantener algunas columnas útiles (take first)
    'Daily AQI Value': 'first' if 'Daily AQI Value' in df.columns else 'first',
    'Percent Complete': 'first' if 'Percent Complete' in df.columns else 'first',
    'Local Site Name': 'first' if 'Local Site Name' in df.columns else 'first',
    'Site Latitude': 'first' if 'Site Latitude' in df.columns else 'first',
    'Site Longitude': 'first' if 'Site Longitude' in df.columns else 'first'
})

In [14]:
# 10) Reindex: crear rango de fechas por site para garantizar continuidad (rellenado NA)
sites = df['Site ID'].unique().tolist()
full = []
for s in sites:
    tmp = df[df['Site ID']==s].set_index('Date').sort_index()
    idx = pd.date_range(tmp.index.min(), tmp.index.max(), freq='D')
    tmp = tmp.reindex(idx)
    tmp['Site ID'] = s
    tmp.index.name = 'Date'
    full.append(tmp.reset_index())
df = pd.concat(full, ignore_index=True).sort_values(['Site ID','Date']).reset_index(drop=True)

In [20]:
# Asegurar que 'Date' es datetime
df['Date'] = pd.to_datetime(df['Date'])

In [21]:
# Ordenamos por Site ID y fecha (importante)
df = df.sort_values(['Site ID', 'Date'])

In [22]:
# Imputamos CO respetando fechas
def interpolate_group(g):
    return (
        g.set_index('Date')['CO']  # usar Date como índice
         .interpolate(method='time')  # interpolar en base al tiempo
         .ffill()  # rellenar hacia adelante
         .bfill()  # rellenar hacia atrás
    )

In [23]:
# 11) Imputar CO hacia adelante/atrás razonablemente (o usa interpolación)
df['CO'] = df.groupby('Site ID', group_keys=False).apply(interpolate_group).reset_index(drop=True)


/tmp/ipython-input-4000200755.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['CO'] = df.groupby('Site ID', group_keys=False).apply(interpolate_group).reset_index(drop=True)


In [24]:
print(df.head())

        Date    Site ID   CO  Daily AQI Value  Percent Complete  \
0 2023-01-01  360050133  0.4              5.0              79.0   
1 2023-01-02  360050133  0.4              5.0             100.0   
2 2023-01-03  360050133  0.8              9.0              75.0   
3 2023-01-04  360050133  0.8              9.0             100.0   
4 2023-01-05  360050133  0.3              3.0             100.0   

   Local Site Name  Site Latitude  Site Longitude  
0  PFIZER LAB SITE        40.8679       -73.87809  
1  PFIZER LAB SITE        40.8679       -73.87809  
2  PFIZER LAB SITE        40.8679       -73.87809  
3  PFIZER LAB SITE        40.8679       -73.87809  
4  PFIZER LAB SITE        40.8679       -73.87809  


In [25]:
# Revisar un sitio específico antes y después de imputar
site = df['Site ID'].iloc[0]
print(df[df['Site ID'] == site][['Date','CO']].head(20))

         Date   CO
0  2023-01-01  0.4
1  2023-01-02  0.4
2  2023-01-03  0.8
3  2023-01-04  0.8
4  2023-01-05  0.3
5  2023-01-06  0.4
6  2023-01-07  0.3
7  2023-01-08  0.3
8  2023-01-09  0.3
9  2023-01-10  0.3
10 2023-01-11  0.3
11 2023-01-12  0.3
12 2023-01-13  0.3
13 2023-01-14  0.2
14 2023-01-15  0.1
15 2023-01-16  0.4
16 2023-01-17  0.6
17 2023-01-18  0.5
18 2023-01-19  0.5
19 2023-01-20  0.4


In [26]:
# 12) Crear features de fecha y lags/rolling
def create_ts_features(df):
    df['dayofweek'] = df['Date'].dt.weekday
    df['day'] = df['Date'].dt.day
    df['month'] = df['Date'].dt.month
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['is_weekend'] = df['dayofweek'].isin([5,6]).astype(int)
    # lags y rolling (por sitio)
    lags = [1,2,3,7,14,30]
    for l in lags:
        df[f'lag_{l}'] = df.groupby('Site ID')['CO'].shift(l)
    # rolling means
    df['rolling_7'] = df.groupby('Site ID')['CO'].shift(1).rolling(window=7, min_periods=1).mean().reset_index(level=0, drop=True)
    df['rolling_30'] = df.groupby('Site ID')['CO'].shift(1).rolling(window=30, min_periods=1).mean().reset_index(level=0, drop=True)
    return df

df = create_ts_features(df)

In [27]:
# 13) Filtrar filas con NA en lags (las primeras filas tendrán NA)
df_model = df.dropna(subset=['lag_1']).copy()  # requiere al menos lag_1

In [28]:
# 14) Columnas útiles para modelar
feature_cols = [c for c in df_model.columns if c not in ['CO','Local Site Name']]
# pero quitamos 'Date'
feature_cols = [c for c in feature_cols if c!='Date']

In [29]:
# 15) Convertir Site ID a categórica con encoding sencillo (LightGBM acepta categoricals por índice)
df_model['Site ID'] = df_model['Site ID'].astype('category')

In [30]:
# 16) División train/valid: entrenamos con todo hasta 2024 (tenemos solo 2023+2024), valid split por tiempo
train = df_model[df_model['Date'] < pd.to_datetime("2024-10-01")]  # ejemplo split por fecha
valid = df_model[df_model['Date'] >= pd.to_datetime("2024-10-01")]

X_train = train[feature_cols]
y_train = train['CO']
X_valid = valid[feature_cols]
y_valid = valid['CO']

In [32]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation

# 17) Entrenar modelo con LightGBM y early stopping
model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    num_boost_round=2000,
    callbacks=[early_stopping(50), log_evaluation(50)]
)

Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.0150101	valid_1's rmse: 0.0185018
[100]	training's rmse: 0.00883877	valid_1's rmse: 0.0115688
[150]	training's rmse: 0.0073473	valid_1's rmse: 0.00927439
[200]	training's rmse: 0.00654731	valid_1's rmse: 0.00842249
[250]	training's rmse: 0.00597182	valid_1's rmse: 0.00782721
[300]	training's rmse: 0.00548924	valid_1's rmse: 0.00738904
[350]	training's rmse: 0.00507838	valid_1's rmse: 0.00704393
[400]	training's rmse: 0.00473293	valid_1's rmse: 0.00682122
[450]	training's rmse: 0.00441975	valid_1's rmse: 0.00658662
[500]	training's rmse: 0.00414946	valid_1's rmse: 0.00644112
[550]	training's rmse: 0.00389448	valid_1's rmse: 0.0062976
[600]	training's rmse: 0.00365217	valid_1's rmse: 0.00615064
[650]	training's rmse: 0.00338501	valid_1's rmse: 0.00603756
[700]	training's rmse: 0.00312971	valid_1's rmse: 0.00588962
[750]	training's rmse: 0.00289837	valid_1's rmse: 0.00571827
[800]	training's rmse: 0.00271

In [34]:
# 18) Validación del modelo
from sklearn.metrics import mean_squared_error
import numpy as np

# Predicciones sobre el conjunto de validación
pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

# Cálculo del RMSE (raíz del error cuadrático medio)
rmse = np.sqrt(mean_squared_error(y_valid, pred_valid))
print("RMSE valid:", rmse)

RMSE valid: 0.004941793426680434
